# DASK DATAFRAME

What I did first was read the official documentation, to see what exactly was recommended to do in Dask’s instead of regular Dataframes. Here are the relevant parts from the official docs:

    Manipulating large datasets, even when those datasets don’t fit in memory
    Accelerating long computations by using many cores
    Distributed computing on large datasets with standard Pandas operations like groupby, join, and time series computations

And then below that, it lists some of the things that are really fast if you use Dask Dataframes:

    Arithmetic operations (multiplying or adding to a Series)
    Common aggregations (mean, min, max, sum, etc.)
    Calling apply (as long as it’s along the index -that is, not after a groupby(‘y’) where ‘y’ is not the index-)
    Calling value_counts(), drop_duplicates() or corr()
    Filtering with loc, isin, and row-wise selection

In [12]:
import pandas as pd
import numpy as np
import dask.dataframe as ddf
import time

In [2]:
df = pd.read_csv('vgsales.csv')

In [3]:
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [4]:
df.shape

(16598, 11)

## How to read file in Dask DataFrame

In [51]:
# 1st Way: (Like Pandas)

dd = ddf.read_csv('vgsales.csv') #if we set blocksize=25e6 then the chunksize would be 25 mb

# 2nd Way: (From existing Pandas DF)

dd2 = ddf.from_pandas(df, npartitions=4) # higher partiton number is better

In [52]:
dd2.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [53]:
%%timeit

df.groupby('Genre')['Platform'].sum()

7.02 ms ± 323 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
%%timeit

dd.groupby('Genre')['Platform'].sum().compute()

51.6 ms ± 3.12 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [55]:
%%timeit

dd2.groupby('Genre')['Platform'].sum().compute()

24.4 ms ± 993 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [56]:
%%timeit

df['NA_Sales'].mean()

56.5 µs ± 1.73 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [57]:
%%timeit
dd['NA_Sales'].mean().compute()

44.4 ms ± 837 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [58]:
%%timeit
dd2['NA_Sales'].mean().compute()

5.42 ms ± 154 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
